In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
from tqdm import tqdm
data = pd.read_csv('./data/spotify-2023_man.csv')
headers = {'content-type': 'application/json', 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
songs = []
data = data['artist.s._name'].to_numpy()
for i in tqdm(range(len(data))):
    artists = data[i].split(', ')
    cumulative_monthly_listeners = 0
    for artist in range(len(artists)):
        except_count = 0 
        while except_count < 10:
            try:
                artist_json = requests.get(f"https://data.songstats.com/api/v1/subscriptions/artist_label_search?q={artists[artist]}&searchType=all&idUnique=r7nmhstl", headers=headers).json()
                artist_json = sorted(artist_json['artists'], key=lambda k: k['tier'])[0]
                id = artist_json['idUnique']
                artist_json = requests.get(f"https://data.songstats.com/api/v1/analytics/multi_entity_stat_overview?sources=overview&forWidget=false&idUnique={id}", headers=headers).json()
                monthly_listeners = artist_json['data'][0]['data']['monthlyListenersCurrent']
                print(artists[artist], id, monthly_listeners)
                cumulative_monthly_listeners += monthly_listeners
                except_count = 11
            except Exception:
                except_count += 1
                print(f'Exception has been caught exception count: {except_count} / 10')
                time.sleep(10)
                continue
    cumulative_monthly_listeners /= len(artists)
    songs.append((i + 1, cumulative_monthly_listeners))
    with open("./data/cumulative_listeners.txt", "a") as f:
        f.write(f'{i},{cumulative_monthly_listeners}\n')
    print(f'Song number {i + 1} is successfully exportet with', cumulative_monthly_listeners, 'monthly listeners')
    #print(f'Actors {actors}')


artist_page = requests.get("https://data.songstats.com/api/v1/subscriptions/artist_label_search?q=Jung Kook&searchType=all&idUnique=r7nmhstl", headers=headers)
artist_json = artist_page.json()
